# Alternating Least Squares (ALS) Algorithm

In [1]:
import numpy as np
import tensorly as tl
from tensorly.tenalg import mode_dot
from numpy.random import randn, rand, randint
from numpy.linalg import pinv, matrix_rank, norm
from scipy.io import loadmat
import time

cpd_tensor = loadmat("cpd_tensor.mat")

In [2]:
def error(X_0, X_0_hat):
    return norm(X_0_hat - X_0, 'fro')**2/norm(X_0, 'fro')**2

def Hermitian(X):
    return np.array(np.matrix(X).H)

def khatri_rao(A, B):
    assert A.shape[1] == B.shape[1]
    
    A = A.T
    B = B.T

    result = []

    for Ai in range(0, len(A)):
        for Bi in range(0, len(B)):
            if Ai == Bi:
                result.append(np.kron(A[Ai], B[Bi]))

    return np.array(result).T 

In [3]:
R = 3
# X = cpd_tensor['tenX']
X = tl.fold(np.matmul(cpd_tensor['A'], khatri_rao(cpd_tensor['B'], cpd_tensor['C']).T), 0, (8, 4, 5))
# X = tl.fold(np.outer([i for i in range(8)], np.outer([i for i in range(4)], [i for i in range(5)])), 0, (8, 4, 5))

In [4]:
print(cpd_tensor['A'].shape)
print(cpd_tensor['B'].shape)
print(cpd_tensor['C'].shape)

(8, 3)
(4, 3)
(5, 3)


In [5]:
print(X.shape)
print(tl.unfold(X, 0).shape)
print(tl.unfold(X, 1).shape)
print(tl.unfold(X, 2).shape)

(8, 4, 5)
(8, 20)
(4, 40)
(5, 32)


## Problem 1

In [6]:
def PARAFAC_ALS(X, R, max_iterations=100, thresh=1e-6):
    A_hat_2 = randn(X.shape[2], R)
    A_hat_3 = randn(X.shape[0], R)

    i = 0
    errs = []
    while i <= max_iterations:
        # A_hat_1
        A_hat_1 = np.matmul(tl.unfold(X, 1), pinv(khatri_rao(A_hat_3, A_hat_2).T))

        # A_hat_2
        A_hat_2 = np.matmul(tl.unfold(X, 2), pinv(khatri_rao(A_hat_3, A_hat_1).T))

        # A_hat_3
        A_hat_3 = np.matmul(tl.unfold(X, 0), pinv(khatri_rao(A_hat_1, A_hat_2).T))
        i += 1
        
        err = norm(tl.unfold(X, 1) - np.matmul(A_hat_1, khatri_rao(A_hat_3, A_hat_2).T), 'fro')
        errs.append(err)
        
        if err <= thresh:
            break
    
    return A_hat_1, A_hat_2, A_hat_3, errs

In [7]:
errs_list = []

for i in range(10):
    A_hat_1, A_hat_2, A_hat_3, errs = PARAFAC_ALS(X, 3, max_iterations=500)

    errs_list.append(errs)
    
# norm(tl.unfold(X, 1) - np.matmul(A_hat_1, khatri_rao(A_hat_3, A_hat_2).T), 'fro')

In [14]:
cpd_tensor['B']

array([[ 0.33235311,  0.76187344,  0.62939088],
       [-1.3645851 ,  0.27145345,  2.86890394],
       [ 0.55350644,  1.83472248, -1.16952832],
       [-0.6355691 ,  0.08686903, -0.66857716]])

In [12]:
A_hat_1

array([[-0.25907768,  0.2751705 ,  0.23718905],
       [-1.1809337 ,  0.09804247, -0.97385751],
       [ 0.4814157 ,  0.66265807,  0.39501853],
       [ 0.27520804,  0.03137508, -0.4535839 ]])

In [20]:
cpd_tensor['B']/(A_hat_1[:, [2, 1, 0]])

array([[ 1.40121612,  2.76873231, -2.42935198],
       [ 1.4012164 ,  2.76873334, -2.42935225],
       [ 1.40121639,  2.7687318 , -2.42935226],
       [ 1.40121618,  2.76872679, -2.42935188]])

In [21]:
for i in cpd_tensor['B']/(A_hat_1[:, [2, 1, 0]]):
    print("{} & {} & {} \\\\".format(
        str(round(i[0], 3)).strip('(').strip(')'), 
        str(round(i[1], 3)).strip('(').strip(')'),
        str(round(i[2], 3)).strip('(').strip(')')))

1.401 & 2.769 & -2.429 \\
1.401 & 2.769 & -2.429 \\
1.401 & 2.769 & -2.429 \\
1.401 & 2.769 & -2.429 \\


In [267]:
import plotly.graph_objects as go



fig = go.Figure(
    layout=go.Layout(
        title="Error - PARAFAC approximation",
        template="simple_white"))

for errs in errs_list:
    fig.add_trace(go.Scatter(x=[i for i in range(500)], y=errs, mode='lines'))

fig.update_yaxes(title='Error',showgrid=True,type="log")
fig.update_xaxes(title='iteration')

fig.show()

## Problem 2

In [215]:
I, J, K = 10, 4, 2

A = randn(I, R) + randn(I, R)*1j
B = randn(J, R) + randn(J, R)*1j
C = randn(K, R) + randn(K, R)*1j

np.matmul(A, khatri_rao(B, C).T).shape

(10, 8)

In [274]:
I, J, K = 10, 4, 2
R = 3

SNRdb_range = [0, 5, 10, 15, 20, 25, 30]
No_experiments = 1000

NMSE = []
NMSE_A = []
NMSE_B = []
NMSE_C = []

for SNRdb in SNRdb_range:
    exp_errors = [] # list of errors for the experiments
    exp_errors_A = [] # list of errors for the experiments
    exp_errors_B = [] # list of errors for the experiments
    exp_errors_C = [] # list of errors for the experiments
    
    for i in range(No_experiments):
        print(f"Experiment {i} with {SNRdb} SNRdb starting...")
        A = randn(I, R) + randn(I, R)*1j
        B = randn(J, R) + randn(J, R)*1j
        C = randn(K, R) + randn(K, R)*1j
        
        X_0 = np.matmul(A, khatri_rao(B, C).T)
        
        V = randn(I, J*K) + randn(I, J*K)*1j
        
        # calculate alpha and X
        alpha = np.sqrt((1/10**(SNRdb/10))*(norm(X_0, 'fro')**2/norm(V, 'fro')**2))
        X = X_0 + alpha*V
        
        # Estimante via MLS-KRF
        An, Bn, Cn, _ = PARAFAC_ALS(tl.fold(X, 0, (I, J, K)), R, max_iterations=300)
        
        X_0 = tl.fold(X_0, 0, (I, J, K))
        
        X_0_hat = np.matmul(An, khatri_rao(Cn, Bn).T)
        
        exp_errors.append(error(tl.unfold(X_0, 1), X_0_hat))
        exp_errors_A.append(error(B, An))
        exp_errors_B.append(error(C, Bn))
        exp_errors_C.append(error(A, Cn))
        print('done')
        
    NMSE.append(np.sum(exp_errors)/No_experiments)
    NMSE_A.append(np.sum(exp_errors_A)/No_experiments)
    NMSE_B.append(np.sum(exp_errors_B)/No_experiments)
    NMSE_C.append(np.sum(exp_errors_C)/No_experiments)

Experiment 0 with 0 SNRdb starting...
done
Experiment 1 with 0 SNRdb starting...
done
Experiment 2 with 0 SNRdb starting...
done
Experiment 3 with 0 SNRdb starting...
done
Experiment 4 with 0 SNRdb starting...
done
Experiment 5 with 0 SNRdb starting...
done
Experiment 6 with 0 SNRdb starting...
done
Experiment 7 with 0 SNRdb starting...
done
Experiment 8 with 0 SNRdb starting...
done
Experiment 9 with 0 SNRdb starting...
done
Experiment 10 with 0 SNRdb starting...
done
Experiment 11 with 0 SNRdb starting...
done
Experiment 12 with 0 SNRdb starting...
done
Experiment 13 with 0 SNRdb starting...
done
Experiment 14 with 0 SNRdb starting...
done
Experiment 15 with 0 SNRdb starting...
done
Experiment 16 with 0 SNRdb starting...
done
Experiment 17 with 0 SNRdb starting...
done
Experiment 18 with 0 SNRdb starting...
done
Experiment 19 with 0 SNRdb starting...
done
Experiment 20 with 0 SNRdb starting...
done
Experiment 21 with 0 SNRdb starting...
done
Experiment 22 with 0 SNRdb starting...
don

done
Experiment 185 with 0 SNRdb starting...
done
Experiment 186 with 0 SNRdb starting...
done
Experiment 187 with 0 SNRdb starting...
done
Experiment 188 with 0 SNRdb starting...
done
Experiment 189 with 0 SNRdb starting...
done
Experiment 190 with 0 SNRdb starting...
done
Experiment 191 with 0 SNRdb starting...
done
Experiment 192 with 0 SNRdb starting...
done
Experiment 193 with 0 SNRdb starting...
done
Experiment 194 with 0 SNRdb starting...
done
Experiment 195 with 0 SNRdb starting...
done
Experiment 196 with 0 SNRdb starting...
done
Experiment 197 with 0 SNRdb starting...
done
Experiment 198 with 0 SNRdb starting...
done
Experiment 199 with 0 SNRdb starting...
done
Experiment 200 with 0 SNRdb starting...
done
Experiment 201 with 0 SNRdb starting...
done
Experiment 202 with 0 SNRdb starting...
done
Experiment 203 with 0 SNRdb starting...
done
Experiment 204 with 0 SNRdb starting...
done
Experiment 205 with 0 SNRdb starting...
done
Experiment 206 with 0 SNRdb starting...
done
Exper

done
Experiment 368 with 0 SNRdb starting...
done
Experiment 369 with 0 SNRdb starting...
done
Experiment 370 with 0 SNRdb starting...
done
Experiment 371 with 0 SNRdb starting...
done
Experiment 372 with 0 SNRdb starting...
done
Experiment 373 with 0 SNRdb starting...
done
Experiment 374 with 0 SNRdb starting...
done
Experiment 375 with 0 SNRdb starting...
done
Experiment 376 with 0 SNRdb starting...
done
Experiment 377 with 0 SNRdb starting...
done
Experiment 378 with 0 SNRdb starting...
done
Experiment 379 with 0 SNRdb starting...
done
Experiment 380 with 0 SNRdb starting...
done
Experiment 381 with 0 SNRdb starting...
done
Experiment 382 with 0 SNRdb starting...
done
Experiment 383 with 0 SNRdb starting...
done
Experiment 384 with 0 SNRdb starting...
done
Experiment 385 with 0 SNRdb starting...
done
Experiment 386 with 0 SNRdb starting...
done
Experiment 387 with 0 SNRdb starting...
done
Experiment 388 with 0 SNRdb starting...
done
Experiment 389 with 0 SNRdb starting...
done
Exper

done
Experiment 552 with 0 SNRdb starting...
done
Experiment 553 with 0 SNRdb starting...
done
Experiment 554 with 0 SNRdb starting...
done
Experiment 555 with 0 SNRdb starting...
done
Experiment 556 with 0 SNRdb starting...
done
Experiment 557 with 0 SNRdb starting...
done
Experiment 558 with 0 SNRdb starting...
done
Experiment 559 with 0 SNRdb starting...
done
Experiment 560 with 0 SNRdb starting...
done
Experiment 561 with 0 SNRdb starting...
done
Experiment 562 with 0 SNRdb starting...
done
Experiment 563 with 0 SNRdb starting...
done
Experiment 564 with 0 SNRdb starting...
done
Experiment 565 with 0 SNRdb starting...
done
Experiment 566 with 0 SNRdb starting...
done
Experiment 567 with 0 SNRdb starting...
done
Experiment 568 with 0 SNRdb starting...
done
Experiment 569 with 0 SNRdb starting...
done
Experiment 570 with 0 SNRdb starting...
done
Experiment 571 with 0 SNRdb starting...
done
Experiment 572 with 0 SNRdb starting...
done
Experiment 573 with 0 SNRdb starting...
done
Exper

done
Experiment 735 with 0 SNRdb starting...
done
Experiment 736 with 0 SNRdb starting...
done
Experiment 737 with 0 SNRdb starting...
done
Experiment 738 with 0 SNRdb starting...
done
Experiment 739 with 0 SNRdb starting...
done
Experiment 740 with 0 SNRdb starting...
done
Experiment 741 with 0 SNRdb starting...
done
Experiment 742 with 0 SNRdb starting...
done
Experiment 743 with 0 SNRdb starting...
done
Experiment 744 with 0 SNRdb starting...
done
Experiment 745 with 0 SNRdb starting...
done
Experiment 746 with 0 SNRdb starting...
done
Experiment 747 with 0 SNRdb starting...
done
Experiment 748 with 0 SNRdb starting...
done
Experiment 749 with 0 SNRdb starting...
done
Experiment 750 with 0 SNRdb starting...
done
Experiment 751 with 0 SNRdb starting...
done
Experiment 752 with 0 SNRdb starting...
done
Experiment 753 with 0 SNRdb starting...
done
Experiment 754 with 0 SNRdb starting...
done
Experiment 755 with 0 SNRdb starting...
done
Experiment 756 with 0 SNRdb starting...
done
Exper

...

done
Experiment 156 with 30 SNRdb starting...
done
Experiment 157 with 30 SNRdb starting...
done
Experiment 158 with 30 SNRdb starting...
done
Experiment 159 with 30 SNRdb starting...
done
Experiment 160 with 30 SNRdb starting...
done
Experiment 161 with 30 SNRdb starting...
done
Experiment 162 with 30 SNRdb starting...
done
Experiment 163 with 30 SNRdb starting...
done
Experiment 164 with 30 SNRdb starting...
done
Experiment 165 with 30 SNRdb starting...
done
Experiment 166 with 30 SNRdb starting...
done
Experiment 167 with 30 SNRdb starting...
done
Experiment 168 with 30 SNRdb starting...
done
Experiment 169 with 30 SNRdb starting...
done
Experiment 170 with 30 SNRdb starting...
done
Experiment 171 with 30 SNRdb starting...
done
Experiment 172 with 30 SNRdb starting...
done
Experiment 173 with 30 SNRdb starting...
done
Experiment 174 with 30 SNRdb starting...
done
Experiment 175 with 30 SNRdb starting...
done
Experiment 176 with 30 SNRdb starting...
done
Experiment 177 with 30 SNRdb 

done
Experiment 335 with 30 SNRdb starting...
done
Experiment 336 with 30 SNRdb starting...
done
Experiment 337 with 30 SNRdb starting...
done
Experiment 338 with 30 SNRdb starting...
done
Experiment 339 with 30 SNRdb starting...
done
Experiment 340 with 30 SNRdb starting...
done
Experiment 341 with 30 SNRdb starting...
done
Experiment 342 with 30 SNRdb starting...
done
Experiment 343 with 30 SNRdb starting...
done
Experiment 344 with 30 SNRdb starting...
done
Experiment 345 with 30 SNRdb starting...
done
Experiment 346 with 30 SNRdb starting...
done
Experiment 347 with 30 SNRdb starting...
done
Experiment 348 with 30 SNRdb starting...
done
Experiment 349 with 30 SNRdb starting...
done
Experiment 350 with 30 SNRdb starting...
done
Experiment 351 with 30 SNRdb starting...
done
Experiment 352 with 30 SNRdb starting...
done
Experiment 353 with 30 SNRdb starting...
done
Experiment 354 with 30 SNRdb starting...
done
Experiment 355 with 30 SNRdb starting...
done
Experiment 356 with 30 SNRdb 

done
Experiment 514 with 30 SNRdb starting...
done
Experiment 515 with 30 SNRdb starting...
done
Experiment 516 with 30 SNRdb starting...
done
Experiment 517 with 30 SNRdb starting...
done
Experiment 518 with 30 SNRdb starting...
done
Experiment 519 with 30 SNRdb starting...
done
Experiment 520 with 30 SNRdb starting...
done
Experiment 521 with 30 SNRdb starting...
done
Experiment 522 with 30 SNRdb starting...
done
Experiment 523 with 30 SNRdb starting...
done
Experiment 524 with 30 SNRdb starting...
done
Experiment 525 with 30 SNRdb starting...
done
Experiment 526 with 30 SNRdb starting...
done
Experiment 527 with 30 SNRdb starting...
done
Experiment 528 with 30 SNRdb starting...
done
Experiment 529 with 30 SNRdb starting...
done
Experiment 530 with 30 SNRdb starting...
done
Experiment 531 with 30 SNRdb starting...
done
Experiment 532 with 30 SNRdb starting...
done
Experiment 533 with 30 SNRdb starting...
done
Experiment 534 with 30 SNRdb starting...
done
Experiment 535 with 30 SNRdb 

done
Experiment 694 with 30 SNRdb starting...
done
Experiment 695 with 30 SNRdb starting...
done
Experiment 696 with 30 SNRdb starting...
done
Experiment 697 with 30 SNRdb starting...
done
Experiment 698 with 30 SNRdb starting...
done
Experiment 699 with 30 SNRdb starting...
done
Experiment 700 with 30 SNRdb starting...
done
Experiment 701 with 30 SNRdb starting...
done
Experiment 702 with 30 SNRdb starting...
done
Experiment 703 with 30 SNRdb starting...
done
Experiment 704 with 30 SNRdb starting...
done
Experiment 705 with 30 SNRdb starting...
done
Experiment 706 with 30 SNRdb starting...
done
Experiment 707 with 30 SNRdb starting...
done
Experiment 708 with 30 SNRdb starting...
done
Experiment 709 with 30 SNRdb starting...
done
Experiment 710 with 30 SNRdb starting...
done
Experiment 711 with 30 SNRdb starting...
done
Experiment 712 with 30 SNRdb starting...
done
Experiment 713 with 30 SNRdb starting...
done
Experiment 714 with 30 SNRdb starting...
done
Experiment 715 with 30 SNRdb 

done
Experiment 874 with 30 SNRdb starting...
done
Experiment 875 with 30 SNRdb starting...
done
Experiment 876 with 30 SNRdb starting...
done
Experiment 877 with 30 SNRdb starting...
done
Experiment 878 with 30 SNRdb starting...
done
Experiment 879 with 30 SNRdb starting...
done
Experiment 880 with 30 SNRdb starting...
done
Experiment 881 with 30 SNRdb starting...
done
Experiment 882 with 30 SNRdb starting...
done
Experiment 883 with 30 SNRdb starting...
done
Experiment 884 with 30 SNRdb starting...
done
Experiment 885 with 30 SNRdb starting...
done
Experiment 886 with 30 SNRdb starting...
done
Experiment 887 with 30 SNRdb starting...
done
Experiment 888 with 30 SNRdb starting...
done
Experiment 889 with 30 SNRdb starting...
done
Experiment 890 with 30 SNRdb starting...
done
Experiment 891 with 30 SNRdb starting...
done
Experiment 892 with 30 SNRdb starting...
done
Experiment 893 with 30 SNRdb starting...
done
Experiment 894 with 30 SNRdb starting...
done
Experiment 895 with 30 SNRdb 

In [281]:
NMSE

[0.6997527277445783,
 0.20242501409510338,
 0.05832349304831527,
 0.018356822522337935,
 0.006294971043804874,
 0.0028947436848662067,
 0.0018545223551670262]

In [280]:
import plotly.graph_objects as go

fig = go.Figure(
    layout=go.Layout(
        title="NMSE vs SNR curve",
        template="simple_white"))

fig.add_trace(go.Scatter(x=SNRdb_range, y=NMSE, mode='lines+markers', name='MLS-KRF'))

fig.update_yaxes(title='NMSE',showgrid=True,type="log")
fig.update_xaxes(title='SNR(dB)')

fig.show()

In [279]:
import plotly.graph_objects as go

fig = go.Figure(
    layout=go.Layout(
        title="NMSE vs SNR curve",
        template="simple_white"))

fig.add_trace(go.Scatter(x=SNRdb_range, y=NMSE_A, mode='lines+markers', name='A'))
fig.add_trace(go.Scatter(x=SNRdb_range, y=NMSE_B, mode='lines+markers', name='B'))
fig.add_trace(go.Scatter(x=SNRdb_range, y=NMSE_C, mode='lines+markers', name='C'))

fig.update_yaxes(title='NMSE',showgrid=True,type="log")
fig.update_xaxes(title='SNR(dB)')

fig.show()